In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dask.distributed import Client, LocalCluster
from dask_jobqueue import SLURMCluster
from dask.diagnostics import ProgressBar

In [3]:
# cluster = SLURMCluster(queue='analysis', cores=4, project='gfdl_o',
#                        memory="24GB", walltime="00:30:00",
#                        scheduler_options={"dashboard_address": "localhost:8726"})
cluster = SLURMCluster(queue='analysis', cores=4, project='gfdl_o',
                       processes=1,
                       memory="64GB", walltime="00:30:00",
                       scheduler_options={"dashboard_address": "localhost:8726"})
# cluster = LocalCluster(memory_limit="256GB",
#                        threads_per_worker=10,
#                        n_workers=1,
#                        dashboard_address=8726,
#                        processes=False)

In [4]:
cluster.scale(8)
client = Client(cluster)
client

Client Scheduler: tcp://140.208.147.171:41243 Dashboard: http://140.208.147.171:8726/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Decadal variability of WMT in OM4 hincast
### Surface fluxes

In [5]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from xgcm import Grid
import glob
import wmt_bgc.basic as wmt
%matplotlib inline

In [6]:
# Specifications for heat and salt budgets
variables = {'heat':'tos','salt':'sos'}
processes=['nonadvective flux','mass flux']
terms = {}
terms['heat'] = {'nonadvective flux':'hfds',
         'mass flux': 'wfo'}
terms['salt'] = {'nonadvective flux':'sfdsi',
         'mass flux':'wfo'}
# Additional variables to load
extra = {}
extra['surface'] = {'mass flux':'wfo'}
extra['full'] = {'volume':'volcello'}

In [7]:
# Load the data
config = 'OM4p25_JRA55do1.4_0netfw_cycle6'
rootdir = '/archive/Raphael.Dussin/xanadu_esm4_20190304_mom6_2019.08.08/'+config+'/gfdl.ncrc4-intel16-prod/pp/'
years = '*'

# Surface fluxes
pp = 'ocean_monthly'
localdir = '/ts/monthly/5yr/'
filenames = []
for variable in variables:
    filenames.append(glob.glob(rootdir+pp+localdir+pp+'.'+years+'.'+variables[variable]+'.nc'))
    for process in processes:
        term = terms[variable][process]
        if term is not None:
            filenames.append(glob.glob(rootdir+pp+localdir+pp+'.'+years+'.'+term+'.nc'))
print('loading surface fluxes')
ds = xr.open_mfdataset(filenames)

# Grid
grid = xr.open_dataset(rootdir+pp+'/'+pp+'.static.nc')

# # gamma_n
# gammadir = '/archive/gam/so_decadal_variability/OM4p25_JRA55do1.4_0netfw_cycle6/'
# localdir = '/'
# ds = xr.merge([ds,xr.open_mfdataset(gammadir+pp+localdir+pp+'.'+years+'.gamma_n.nc')['gamma_n']])
# ds['gamma_n'] = ds['gamma_n'].where(ds['gamma_n']>0,np.nan)

# Specify some constants
Cp = 3992.
rho0 = 1035.

loading surface fluxes


In [8]:
# Build an xgcm grid object
# Create a Z dimension
grid['z_l'] = np.array([2.5])#xr.DataArray(np.array([2.5]),dims=['z_l'],coords={'z_l':np.array([2.5])})
grid['z_i'] = np.array([0,5])
grid['dzt'] = xr.DataArray(np.array([5]),dims=['z_l'],coords={'z_l':grid['z_l']})
# grid = grid.squeeze() # Get rid of any remnant time variables

# Fill in nans with zeros
grid['dxt'] = grid['dxt'].fillna(0.)
grid['dyt'] = grid['dyt'].fillna(0.)
grid['dzt'] = grid['dzt'].fillna(0.)
grid['areacello'] = grid['areacello'].fillna(0.)
grid['volcello'] = grid['areacello']*grid['dzt'].fillna(0.)
metrics = {
    ('X',): ['dxt','dxCu','dxCv'], # X distances
    ('Y',): ['dyt','dyCu','dyCv'], # Y distances
    ('Z',): ['dzt'], # Z distances
    ('X', 'Y'): ['areacello'], # Areas
    ('X', 'Y', 'Z'): ['volcello'], # Volumes
}
coords={'X': {'center': 'xh', 'right': 'xq'},
        'Y': {'center': 'yh', 'right': 'yq'},
        'Z': {'center': 'z_l', 'outer': 'z_i'} }
xgrid = Grid(grid, coords=coords, metrics=metrics, periodic=['X'])

In [9]:
# Place all the surface fluxes at the surface interface
ds = ds.expand_dims({'z_l':grid['z_l']})

### Prior calculations

In [10]:
# Pressure from depth
ds['p'] = wmt.gsw_p_from_z(-grid['z_l'][0],grid['geolat'])
ds['alpha'] = wmt.gsw_alpha(ds['sos'],ds['tos'],ds['p'])
ds['beta'] = wmt.gsw_beta(ds['sos'],ds['tos'],ds['p'])
ds['sigma0'] = wmt.gsw_sigma0(ds['sos'],ds['tos'])

# Tracer flux due to mass flux at ocean surface
for variable in variables:
    vari = variables[variable]
    if variable=='heat':
        Jlmass = ds[extra['surface']['mass flux']]*(ds[vari]-ds[vari])
    elif variable=='salt':
        Jlmass = ds[extra['surface']['mass flux']]*(ds[vari]-xr.zeros_like(ds[vari]))
    ds[vari+'_massflux'] = Jlmass

### Calculate $h\dot{\lambda}$ and $h\dot{\rho}$

In [11]:
hldot={}
hrhodot={}
for variable in variables:
    hldot[variable]=xr.Dataset()
    hrhodot[variable]=xr.Dataset()
    if variable=='heat':
        drhodl = -ds['alpha']
    elif variable=='salt':
        drhodl = ds['beta']
    vari = variables[variable]
    for process in processes:
        term = terms[variable][process]
        if term is not None:
            hldot[variable][process] = ds[term]
            if variable=='salt': # Multiply by 1000 to convert to gkg-1
                hldot[variable][process]*=1000
            if variable=='heat': # Divide by Cp to get temperature tendency
                hldot[variable][process]/=Cp
            if process=='mass flux':
                hldot[variable][process]=ds[vari+'_massflux']
            hrhodot[variable][process] = drhodl*hldot[variable][process]

# Sum up contributions to total tendency
# This is hacky, necessarily assumes that process not present in salt present in heat
hrhodot['total'] = xr.Dataset()
for process in processes:
    if process in hrhodot['heat'].data_vars:
        hrhodot['total'][process]=hrhodot['heat'][process]
    if process in hrhodot['salt'].data_vars:
        hrhodot['total'][process]+=hrhodot['salt'][process]

### Apply precursors, select subregion, and transform

In [12]:
# bins
bins=np.arange(26,28.5,0.1)
ds['sigma0_i'] = xgrid.interp(ds['sigma0'],'Z',boundary='extend').chunk({'z_i': -1})
ds['b'] = xr.ones_like(ds['sigma0']) # HACK
# Select subregion
selection = {'yh':slice(-90,-30)}
ds_subregion = ds.sel(selection)
# Precursors
precursor = (ds_subregion['sigma0']+1000)*ds_subregion['b']

#
variables_now = ['heat','salt','total']

hrhodot_on_gamma = {}
for variable in variables_now:
    hrhodot_subregion = hrhodot[variable].sel(selection)
    hrhodot_on_gamma[variable]=xr.Dataset()
    for process in processes:
        if process in hrhodot_subregion.data_vars:
            hrhodot_on_gamma[variable][process] = xgrid.transform(
                precursor*hrhodot_subregion[process],'Z',target=bins,target_data=ds_subregion['sigma0_i'],method='conservative')

/nbhome/gam/miniconda/envs/mom6-clean/lib/python3.8/site-packages/xgcm-0.4.0+18.ge0eae1e-py3.8.egg/xgcm/transform.py:227: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
/nbhome/gam/miniconda/envs/mom6-clean/lib/python3.8/site-packages/xgcm-0.4.0+18.ge0eae1e-py3.8.egg/xgcm/transform.py:227: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
/nbhome/gam/miniconda/envs/mom6-clean/lib/python3.8/site-packages/xgcm-0.4.0+18.ge0eae1e-py3.8.egg/xgcm/transform.py:227: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
/nbhome/gam/miniconda/envs/mom6-clean/lib/python3.8/site-packages/xgcm-0.4.0+18.ge0eae1e-py3.8.egg/xgcm/transform.py:227: FutureWarning: ``output_sizes`` should be given in the ``dask_guf

### Calculate G

In [13]:
G = {}
for variable in variables_now:
    G[variable] = xr.Dataset()
    for process in processes:
        if process in hrhodot_on_gamma[variable].data_vars:
            G[variable][process] = (hrhodot_on_gamma[variable][process]*grid['areacello']).sum(['xh','yh'])/np.diff(bins)

In [ ]:
%%time
G.load()

In [14]:
%%time
outdir = '/work/gam/projects/so_decadal_variability/data/processed/'+config+'/'
for variable in variables_now:
    G[variable].to_netcdf(outdir+'G_'+variable+'.nc')

/nbhome/gam/miniconda/envs/mom6-clean/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/nbhome/gam/miniconda/envs/mom6-clean/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/nbhome/gam/miniconda/envs/mom6-clean/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


CPU times: user 25min 4s, sys: 28min 20s, total: 53min 25s
Wall time: 12min 54s
